In [1]:
import chazutsu

r = chazutsu.datasets.MovieReview.polarity().download(force=True, test_size=0.0)

Make directory for downloading the file to /Users/smap3/PycharmProjects/naive_bayse_anatomy.


Begin downloading the Moview Review Data dataset from http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz.


The dataset file is saved to /Users/smap3/PycharmProjects/naive_bayse_anatomy/data/moview_review_data/review_polarity.tar.gz


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


...

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.



Shuffle the extracted dataset.


Done all process! Make below files at /Users/smap3/PycharmProjects/naive_bayse_anatomy/data/moview_review_data


 review_polarity.txt


# CountVectorizer.fit_transform

Learn the vocabulary dictionary and return term-document matrix.

単語を抽出し、ボキャブラリ(単語の一覧)として行列で返す。





In [24]:
from sklearn.feature_extraction.text import CountVectorizer

# BoWに変換する
count_vect = CountVectorizer(token_pattern=r'[A-Za-z_]+')
X_train_counts = count_vect.fit_transform(r.data().review)

# 疎行列(csr-matrix)となっている。内容確認する。
f = open('X_train_counts.txt', 'w')

# dataは文書における各単語の出現回数
for idx in range(len(X_train_counts.data)):
    f.writelines(str(X_train_counts.data[idx]) + " ")
f.writelines(str("\n"))

# indicesは出現数が0ではない何列目の要素(単語)かを表す
for idx in range(len(X_train_counts.indices)):
    f.writelines(str(X_train_counts.indices[idx]) + " ")
f.writelines(str("\n"))

# indptrはどこまで同一の行(文書)かを表す
for idx in range(len(X_train_counts.indptr)):
    f.writelines(str(X_train_counts.indptr[idx]) + " ")
f.writelines(str("\n"))

f.close()

今回の訓練データ全体の単語数は39204種類。

In [25]:
len(count_vect.vocabulary_)

39204

#

一つ目の文書にスポットを当てて確認する。
単語の種類と出現数を確認。


In [34]:
# dictにおける値からkeyを取得する関数。vocaburaly_から単語idを用いて単語名を取得するのに用いる
def voc_find(items, term_id):
    voc = ""
    for key, value in items:
        if value == term_id:
            voc = key
            break
    return voc


# 1つ目の文書の単語の種類と出現数を書き出し
f = open('first_document_vocabulary.txt', 'w')

for ptr in range(X_train_counts.indptr[1]):    
    voc = voc_find(count_vect.vocabulary_.items(), X_train_counts.indices[ptr])

    # 単語id 単語名 出現回数を出力
    f.writelines(str(X_train_counts.indices[ptr]) + " " + str(voc) + " " + str(X_train_counts.data[ptr]) + "\n")

f.close()

tfidfで重み付けが行われた結果の確認。



In [36]:
from sklearn.feature_extraction.text import TfidfTransformer

# tfidfによる重み付け処理
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# 1つ目の文書の単語の種類とf値を書き出し
f = open('first_X_train_tfidf.txt', 'w')
for ptr in range(X_train_tfidf.indptr[1]):
    voc = voc_find(count_vect.vocabulary_.items(), X_train_tfidf.indices[ptr])
    
    # 単語id 単語名 f値を出力
    f.writelines(str(X_train_tfidf.indices[ptr]) + " " + str(voc) + " " + str(X_train_tfidf.data[ptr]) + "\n")

f.close()

ナイーブベイズ分類を実行しつつ、各パラメータを確認

feature_count_  
class_count_  
class_log_prior_  
feature_log_prob_  

In [37]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, r.data().polarity)

# 単語毎に各クラスにおけるの尤度を取得
f = open('feature_log_prob_.txt', 'w')
for i in range(len(clf.feature_log_prob_[0])):
    voc = voc_find(count_vect.vocabulary_.items(), i)

    # 単語名 ネガティブクラスにおける各単語の条件付き確率 ポジティブクラスにおける各単語の条件付き確率 
    f.writelines(voc + " " + str(clf.feature_log_prob_[0][i]) + " " + str(clf.feature_log_prob_[1][i]) + "\n") 
f.close()

この後予測する際は以下となる。

predicted = clf.predict(X_new_tfidf)

ここからpredictの処理を見ていく。
predictは以下の通り。

        jll = self._joint_log_likelihood(X)
        return self.classes_[np.argmax(jll, axis=1)]

_joint_log_likelihoodで各クラスの確率を算出し、
そのうち最も高いクラスを推定クラスとしている。



_joint_log_likelihoodの処理は以下の通り。

        check_is_fitted(self, "classes_")

        X = check_array(X, accept_sparse='csr')
        return (safe_sparse_dot(X, self.feature_log_prob_.T) +
                self.class_log_prior_)

確率を算出している以下処理の確認。

safe_sparse_dot(X, self.feature_log_prob_.T) + self.class_log_prior_


In [11]:
#単語×クラスの行列
print(clf.feature_log_prob_.T)

clf.feature_log_prob_.T.shape

[[-10.12919246 -10.80435366]
 [-10.79022431 -10.87206429]
 [-10.79022431 -10.87206429]
 ..., 
 [-10.81865448 -10.81494686]
 [-10.87230098 -10.746948  ]
 [-10.81170821 -10.87206429]]


(39204, 2)

文書毎に各クラスの推定確率を算出する。

文書中出現単語と単語毎のベイズ確率の行列積を求める。


In [38]:
#safe_sparse_dot(X_train_tfidf, clf.feature_log_prob_.T)

from sklearn.utils.extmath import safe_sparse_dot
from scipy.sparse import issparse


def ssd(a, b, dense_output=False):
    if issparse(a) or issparse(b):
        ret = a * b
        if dense_output and hasattr(ret, "toarray"):
            ret = ret.toarray()
        return ret
    
    else:
        return fast_dot(a, b)

safe_sparse_dot_value = ssd(X_train_tfidf, clf.feature_log_prob_.T)

safe_sparse_dot_value.shape

(2000, 2)

各クラスの事前確率がclass_log_prior_に格納されています。
今回はpositive,negativeともに1000文書ずつ、合計2000文書を訓練データとして与えています。
それらの確率をlogで算出するとその通りになっていることが分かります。



In [18]:
print("class_log_prior_: " + str(clf.class_log_prior_))

print(math.log(0.5))

class_log_prior_: [-0.69314718 -0.69314718]
-0.6931471805599453


array([[-123.45103795, -122.91073791],
       [-132.35858883, -131.94731774],
       [-137.12199247, -136.38472042],
       ..., 
       [-138.57355562, -138.28937654],
       [ -83.26333521,  -82.21486293],
       [-101.51080321, -100.61101538]])

ベイズの定理はP(Y)P(X|Y)だけど、logを取っているので和で求めることができる。
文書毎に各クラスの条件付き確率を算出。
そしてそのうち確率が高い方を推定クラスとする。


In [20]:
score = safe_sparse_dot_value + clf.class_log_prior_
score

array([[-123.45103795, -122.91073791],
       [-132.35858883, -131.94731774],
       [-137.12199247, -136.38472042],
       ..., 
       [-138.57355562, -138.28937654],
       [ -83.26333521,  -82.21486293],
       [-101.51080321, -100.61101538]])

実際に関数を使ったものと比較してみる

In [21]:
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import check_array


check_is_fitted(clf, "classes_")
X_train_tfidf = check_array(X_train_tfidf, accept_sparse='csr')

clf._joint_log_likelihood(X_train_tfidf)

array([[-123.45103795, -122.91073791],
       [-132.35858883, -131.94731774],
       [-137.12199247, -136.38472042],
       ..., 
       [-138.57355562, -138.28937654],
       [ -83.26333521,  -82.21486293],
       [-101.51080321, -100.61101538]])

In [ ]:
完全に一致していることが分かる。

In [23]:
import numpy as np

np.mean(score == clf._joint_log_likelihood(X_train_tfidf))

1.0